In [202]:
from pymongo import MongoClient
import time
import pandas as pd

articles = MongoClient()['pmc']['articles']
geonames = MongoClient()['geonames']['allCountries']

articles.count({'index.infectious': 1})

articles.count({'meta.article-type': 'research-article'})

articles.count({'index.infectious': 1, 'meta.article-type': 'research-article'})

cursor = articles.find({'index.infectious': 1, 'meta.article-type': 'research-article'})

projection = {'_id': 0, 'geonameid': 1, 'name': 1, 'latitude': 1, 'longitude': 1, 'population': 1}

def geoname_dataframe(article, projection=None):
    if projection is None:
        projection = {'_id': 0, 'geonameid': 1, 'name': 1, 'latitude': 1, 'longitude': 1, 'population': 1}
    df = pd.DataFrame()
    if len(article['geonames']['culled']) == 0: # In the future, this should raise an error
        return(df)
    for loc in article['geonames']['culled']:
        row = geonames.find_one({'geonameid': loc['geonameid']}, projection)
        df = df.append(row, ignore_index=True)
    # Count multiple mentions and add a column for that, then drop duplicates.
    df['count'] = df.groupby('geonameid')['geonameid'].transform(pd.Series.value_counts)
    df = df.drop_duplicates()
    # Have a column to ID.
    df['article_id'] = article['_id']
    return(df)